In [1]:
# Basis-Setup für Datenanalyse und Embedding
import json
import pandas as pd
import numpy as np
import re
from typing import List, Dict
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import warnings
import tiktoken
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
# Import der Funktionen aus text_to_dqr.py
from text_to_dqr import text_to_dqr, system_prompt
from utils import count_tokens


warnings.filterwarnings("ignore")

/Users/hxx/Library/CloudStorage/OneDrive-Persönlich/Dokumente/Master/2. Semester/Web Mining/web_mining_lm/pythonProject1/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/hxx/Library/CloudStorage/OneDrive-Persönlich/Dokumente/Master/2. Semester/Web Mining/web_mining_lm/pythonProject1/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(system_prompt)


    Du bekommst kurze biographische Texte von Politikern. Diese sind aus Wikipedia-Artikeln extrahiert.
    Deine Aufgabe ist es, aus dem Text den höchsten erreichten Bildungsabschluss nach dem Deutschen Qualifikationsrahmen (DQR) zu erkennen und zurückzugeben.
    
    AUSGABEFORMAT (streng!):
    - Entweder exakt: DQR-Niveau; Sehr kurzer Kommentar zum höchsten Bildungsabschluss; Confidence
      Beispiele:
      8; Dr. der Wirtschaftspsychologie; 5
      7; Volljurist mit 2. Staatsexamen; 4
    - ODER (nur wenn keinerlei bildungsrelevante Information in ALLEN Abschnitten vorhanden ist):
      0; Keine Angabe zum Bildungsabschluss (vermutlich falsches Retrieval); -
    
    Hinweis zum Kommentar:
    - Der Kommentar bleibt sehr kurz. Für 0-Fälle gib einen knappen Grund an (z. B. „vermutlich falsches Retrieval“ oder „explizit: ohne Abschluss/Abbruch“).
    
    Confidence-Score (Definition):
    C=5 (sehr sicher): Explizite Nennung des Abschlusses/Grades/Titels (z. B. „2. Staatsexamen

In [3]:
def load_json(path='testdata/filtered_minister_with_content.json'):
    """Lädt JSON-Daten der Politiker-Biografien"""
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    print(f"Anzahl geladener Politiker: {len(data)}")
    return data

# Beispiel ausführen
data = load_json()

Anzahl geladener Politiker: 165


In [4]:
def extract_content_sections(person):
    """Extrahiert alle Content-Abschnitte von einem Politiker"""
    all_sections = []  

    for content in person.get('neo4j_content', []):
        all_sections.append({
            'politician_name': f"{person['Vorname']} {person['Nachname']}",
            'content_id': content['content_id'],
            'section_content': content['section_content']
        })

    print(f"Gesamtanzahl Abschnitte: {len(all_sections)}")
    return all_sections

# Beispiel ausführen
person = next(item for item in data if item["ID"] == "#091")

example_sections = extract_content_sections(person)
print(example_sections)

Gesamtanzahl Abschnitte: 2
[{'politician_name': 'Christian Schmidt', 'content_id': '33e433f7bd723c61061b7277e0f47cc554cd76c3', 'section_content': '1981 wechselte Schmidt zur Grün-Alternativen Liste (GAL).\nEr gehörte dem Bundestag in der 10. Legislaturperiode an und war unter anderem Sprecher der grünen Bundestagsfraktion. Allerdings war das Bundeshaus auf Rollstuhlfahrer nicht eingerichtet. Schmidt konnte nicht vor dem Rednerpult sprechen, weil das nicht verstellbar war. Seine Reden konnten nicht, wie üblich, vom Hauskanal übertragen werden, weil die dazugehörende Kamera fest auf das Pult ausgerichtet war. Zusammen mit Jutta Ditfurth und Regina Michalik trat Christian Schmidt im Dezember 1988 als Sprecher der grünen Bundespartei zurück. Grund dafür war die fehlende Unterstützung der Bundesversammlung auf dem außerordentlichen Parteitag in Karlsruhe. Zwei Jahre später verließ er die Partei.'}, {'politician_name': 'Christian Schmidt', 'content_id': '4ed89e8199e32030ab23f20e92635e9d03578

In [5]:
def chunk_content_sections(sections, max_tokens=1000, overlap_tokens=200):
    """Chunkt Content-Sections die zu lang sind mit Overlap"""
    chunked_sections = []
    
    for section in sections:
        content = section['section_content']
        token_count = count_tokens(content)
        
        if token_count <= max_tokens:
            # Section ist kurz genug - einfach übernehmen
            chunked_sections.append(section)
        else:
            # Section ist zu lang - aufteilen
            words = content.split()
            current_chunk = []
            current_tokens = 0
            chunk_counter = 0
            
            for word in words:
                word_tokens = count_tokens(word + " ")
                current_chunk.append(word)
                current_tokens += word_tokens
                
                if current_tokens >= max_tokens:
                    # Chunk ist voll - speichern
                    chunk_text = " ".join(current_chunk)
                    chunked_sections.append({
                        'politician_name': section['politician_name'],
                        'content_id': f"#{chunk_counter:02d}_{section['content_id']}",
                        'section_content': chunk_text
                    })
                    
                    # Overlap für nächsten Chunk vorbereiten
                    overlap_words = []
                    overlap_tokens_count = 0
                    for word in reversed(current_chunk):
                        word_tokens = count_tokens(word + " ")
                        if overlap_tokens_count + word_tokens <= overlap_tokens:
                            overlap_words.insert(0, word)
                            overlap_tokens_count += word_tokens
                        else:
                            break
                    
                    current_chunk = overlap_words
                    current_tokens = overlap_tokens_count
                    chunk_counter += 1
            
            # Letzten Chunk hinzufügen falls noch Inhalt vorhanden
            if current_chunk:
                chunk_text = " ".join(current_chunk)
                chunked_sections.append({
                    'politician_name': section['politician_name'],
                    'content_id': f"#{chunk_counter:02d}_{section['content_id']}",
                    'section_content': chunk_text
                })
    
    return chunked_sections

# Nach extract_content_sections
chunked_sections = chunk_content_sections(example_sections)
print(f"Original: {len(example_sections)} Sections")
print(f"Nach Chunking: {len(chunked_sections)} Sections")
print(chunked_sections)

Original: 2 Sections
Nach Chunking: 2 Sections
[{'politician_name': 'Christian Schmidt', 'content_id': '33e433f7bd723c61061b7277e0f47cc554cd76c3', 'section_content': '1981 wechselte Schmidt zur Grün-Alternativen Liste (GAL).\nEr gehörte dem Bundestag in der 10. Legislaturperiode an und war unter anderem Sprecher der grünen Bundestagsfraktion. Allerdings war das Bundeshaus auf Rollstuhlfahrer nicht eingerichtet. Schmidt konnte nicht vor dem Rednerpult sprechen, weil das nicht verstellbar war. Seine Reden konnten nicht, wie üblich, vom Hauskanal übertragen werden, weil die dazugehörende Kamera fest auf das Pult ausgerichtet war. Zusammen mit Jutta Ditfurth und Regina Michalik trat Christian Schmidt im Dezember 1988 als Sprecher der grünen Bundespartei zurück. Grund dafür war die fehlende Unterstützung der Bundesversammlung auf dem außerordentlichen Parteitag in Karlsruhe. Zwei Jahre später verließ er die Partei.'}, {'politician_name': 'Christian Schmidt', 'content_id': '4ed89e8199e32030a

In [6]:
device = "cpu"
model = SentenceTransformer(
    "Qwen/Qwen3-Embedding-0.6B",
    trust_remote_code=True,
    device=device
)

# HARTE KAPPUNG gegen 32k
model.max_seq_length = 1024
model.tokenizer.model_max_length = 1024
print(f"Modell geladen: {model}")

def embed_sections(sections, model, batch_size=16):
    """Erstellt Embeddings für Abschnittstexte mit dem gewählten Modell"""

    texts = [section['section_content'] for section in sections]
    embeddings = model.encode(texts, show_progress_bar=True, convert_to_tensor=False, batch_size=batch_size)

    return np.array(embeddings)

Modell geladen: SentenceTransformer(
  (0): Transformer({'max_seq_length': 1024, 'do_lower_case': False, 'architecture': 'Qwen3Model'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': True, 'include_prompt': True})
  (2): Normalize()
)


In [7]:
section_embeddings = embed_sections(chunked_sections, model)

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.00it/s]


In [8]:
education_queries = [
    "Hat die Person studiert? Welche Schule hat sie besucht? Was ist über ihren Bildungsweg bekannt? Welchen Beruf hat die Person vorher ausgeübt? Hat die Person eine Ausbildung gemacht?"
]

# Queries einbetten
query_embedding = model.encode(education_queries)

In [9]:
# Funktion zum Finden der Top-K ähnlichsten Abschnitte
def find_top_k_sections(query_embedding, section_embeddings, all_sections, top_k=5):
    """Findet die ähnlichsten Content-Abschnitte für eine Query"""
    similarities = cosine_similarity(query_embedding, section_embeddings)[0]
    
    # Top-k Indizes finden
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    results = []
    for idx in top_indices:
        results.append({
            'section': all_sections[idx],  # Hier ist der Fehler - 'section' fehlt
            'similarity': similarities[idx],
            'content_preview': all_sections[idx]['section_content'][:300] + "..."
        })
    
    return results

In [10]:
# Beispiel-Query: Bildungsweg
top_sections = find_top_k_sections(query_embedding, section_embeddings, chunked_sections, top_k=100)

# In DataFrame umwandeln für strukturierte Darstellung
df = pd.DataFrame([{
    "Rang": i+1,
    "Politiker": result['section']['politician_name'],
    "Score": f"{result['similarity']:.3f}",
    "token_count": count_tokens(result['section']['section_content']),
    "Inhalt": result['section']['section_content']
} for i, result in enumerate(top_sections)])

# Lange Spalteninhalte anzeigen
pd.set_option('display.max_colwidth', None)

# Anzeige
display(df)

,Rang,Politiker,Score,token_count,Inhalt
0,1,Christian Schmidt,0.121,212,"Schmidt, der durch einen Sportunfall auf einen Rollstuhl angewiesen war, trat 1965 in die SPD ein. Er zählte zur inoffiziellen Linksfraktion innerhalb der Hamburger SPD und galt als einer der strategischen Köpfe. Als Kandidat der Jungsozialisten wurde er mit einem Drittel der Stimmen in den erweiterten Landesvorstand gewählt. Auf diesem Parteitag hatte es scharfe Auseinandersetzungen zwischen „Rechts“ und „Links“ gegeben. Er war unter anderem Vorsitzender der Kommission Ausländerpolitik und im Landesvorstand für den Bereich „politische Bildung“ zuständig.[2] 1981 trat Schmidt aus der SPD aus. Der eher dem rechten Flügel der Partei zugerechnete SPD-Landesvorsitzende Oswald Paulig bedauerte den Austritt und würdigte die menschlichen Qualitäten von Schmidt."
1,2,Christian Schmidt,0.014,209,"1981 wechselte Schmidt zur Grün-Alternativen Liste (GAL).\nEr gehörte dem Bundestag in der 10. Legislaturperiode an und war unter anderem Sprecher der grünen Bundestagsfraktion. Allerdings war das Bundeshaus auf Rollstuhlfahrer nicht eingerichtet. Schmidt konnte nicht vor dem Rednerpult sprechen, weil das nicht verstellbar war. Seine Reden konnten nicht, wie üblich, vom Hauskanal übertragen werden, weil die dazugehörende Kamera fest auf das Pult ausgerichtet war. Zusammen mit Jutta Ditfurth und Regina Michalik trat Christian Schmidt im Dezember 1988 als Sprecher der grünen Bundespartei zurück. Grund dafür war die fehlende Unterstützung der Bundesversammlung auf dem außerordentlichen Parteitag in Karlsruhe. Zwei Jahre später verließ er die Partei."


In [11]:
biography = df.iloc[0,4] + df.iloc[1,4] #+ df.iloc[2,4]
#print(system_prompt)
print(text_to_dqr("gpt-4.1", system_prompt, biography))

(0, 'Keine Angabe zum Bildungsabschluss (vermutlich falsches Retrieval)', None, 3265, 17, 0)


In [12]:
biography

'Schmidt, der durch einen Sportunfall auf einen Rollstuhl angewiesen war, trat 1965 in die SPD ein. Er zählte zur inoffiziellen Linksfraktion innerhalb der Hamburger SPD und galt als einer der strategischen Köpfe. Als Kandidat der Jungsozialisten wurde er mit einem Drittel der Stimmen in den erweiterten Landesvorstand gewählt. Auf diesem Parteitag hatte es scharfe Auseinandersetzungen zwischen „Rechts“ und „Links“ gegeben. Er war unter anderem Vorsitzender der Kommission Ausländerpolitik und im Landesvorstand für den Bereich „politische Bildung“ zuständig.[2] 1981 trat Schmidt aus der SPD aus. Der eher dem rechten Flügel der Partei zugerechnete SPD-Landesvorsitzende Oswald Paulig bedauerte den Austritt und würdigte die menschlichen Qualitäten von Schmidt.1981 wechselte Schmidt zur Grün-Alternativen Liste (GAL).\nEr gehörte dem Bundestag in der 10. Legislaturperiode an und war unter anderem Sprecher der grünen Bundestagsfraktion. Allerdings war das Bundeshaus auf Rollstuhlfahrer nicht e

In [15]:
# Schnelle Ausführung:
person = next(item for item in data if item["ID"] == "#002")
example_sections = extract_content_sections(person)
chunked_sections = chunk_content_sections(example_sections)
print(f"Original: {len(example_sections)} Sections")
print(f"Nach Chunking: {len(chunked_sections)} Sections")
section_embeddings = embed_sections(chunked_sections, model)
top_sections = find_top_k_sections(query_embedding, section_embeddings, chunked_sections, top_k=100)

# In DataFrame umwandeln für strukturierte Darstellung
df = pd.DataFrame([{
    "Rang": i+1,
    "Politiker": result['section']['politician_name'],
    "Score": f"{result['similarity']:.3f}",
    "token_count": count_tokens(result['section']['section_content']),
    "Inhalt": result['section']['section_content']
} for i, result in enumerate(top_sections)])

# Lange Spalteninhalte anzeigen
pd.set_option('display.max_colwidth', None)

# Anzeige
display(df)

biography = df.iloc[0,4] + df.iloc[1,4] + df.iloc[2,4] + df.iloc[3,4] #+ df.iloc[4,4] + df.iloc[5,4]
#print(system_prompt)
print(text_to_dqr("gpt-4.1", system_prompt, biography))

Gesamtanzahl Abschnitte: 5
Original: 5 Sections
Nach Chunking: 5 Sections


Batches: 100%|██████████| 1/1 [00:03<00:00,  3.38s/it]


,Rang,Politiker,Score,token_count,Inhalt
0,1,Hans Klein,0.181,13,Seit 1972 war er Mitglied der CSU.
1,2,Hans Klein,0.164,472,"Klein kam 1945 als Heimatvertriebener aus dem Sudetenland nach Heidenheim an der Brenz, wo er die Volks- und Realschule besuchte. Nach der Mittleren Reife absolvierte Klein eine Schriftsetzerlehre sowie ein Zeitungsvolontariat. 1950 erhielt er ein Stipendium für die Fächer Volkswirtschaftslehre und Geschichte am Cooperative College der Universität Leicester in England. Von 1953 bis 1959 war er als Journalist tätig; bis 1956 als Redakteur bei der Heidenheimer Zeitung, daraufhin als Bonner Korrespondent der Nachrichtenagentur DIMITAG (bis 1958) bzw. des Hamburger Abendblatts. 1959 trat er in den Auswärtigen Dienst ein. Er war in dieser Zeit bis 1964 als Presseattaché an den deutschen Botschaften in Jordanien, Syrien, Irak und Indonesien eingesetzt. 1965 wurde er pressepolitischer Referent bei Bundeskanzler Ludwig Erhard. Ab 1968 fungierte er als Pressechef der Olympischen Sommerspiele 1972. Seit 1972 war er als freier Journalist tätig. 1976 verfasste er als Ghostwriter Erhards Memoiren Erfahrungen für die Zukunft. Meine Kanzlerzeit, die dann erst 2024 erschienen.[2] 1990 kandidierte er für den Posten des Münchner Oberbürgermeisters, unterlag jedoch bereits im ersten Wahlgang dem Amtsinhaber Georg Kronawitter deutlich.\nKlein lebte in München. Am 7. November 1996 erlitt er während einer Fahrt mit dem Nachtzug von München nach Bonn einen Herzinfarkt, in dessen Folge er trotz unverzüglicher ärztlicher Versorgung ins Koma fiel und am 26. November 1996 verstarb.\nKlein war verheiratet und hatte drei Kinder. Seine Grabstätte befindet sich auf dem Friedhof Bernau am Chiemsee."
2,3,Hans Klein,0.156,55,"Seit 2002 ist eine Straße im Münchner Stadtteil Sendling nach ihm benannt. Klein war Träger des Bundesverdienstkreuzes, des Bayerischen Verdienstordens und zahlreicher ausländischer Orden."
3,4,Hans Klein,0.084,374,"Hans Klein zwischen Theo Waigel und Helmut Kohl am 18. November 1989 auf dem CSU-Parteitag in München\nNach der Bundestagswahl 1987 wurde er am 12. März 1987 als Bundesminister für wirtschaftliche Zusammenarbeit in die von Bundeskanzler Helmut Kohl geführte Bundesregierung berufen. Nach der Kabinettsumbildung im Frühjahr 1989 übernahm er am 21. April 1989 im Range eines Bundesministers für besondere Aufgaben die Leitung des Presse- und Informationsamtes der Bundesregierung. Nach der Bundestagswahl 1990 schied er im Dezember 1990 aus der Bundesregierung aus und übernahm bis zu seinem Tode das Amt eines Vizepräsidenten des Deutschen Bundestages.\nAls Sprecher der Bundesregierung vertrat Bundesminister Klein die Ansicht, die Angehörigen der Waffen-SS hätten schlicht geglaubt, ihr Vaterland verteidigen zu müssen. Die in einem Interview mit der Boulevard-Zeitschrift Quick am 2. Mai 1989 geäußerte Meinung wiederholte er bei der nächsten Pressekonferenz.[3] Dies löste zwei Anträge von den Fraktionen der SPD und der Grünen aus, die im Wesentlichen eine Missbilligung von Kleins Äußerungen durch den Bundestag erwarteten. Die Debatte darüber wurde am 15. Juni 1989 im Plenum geführt. Klein entschuldigte sich dabei nicht; die Anträge der Opposition wurden in nicht-namentlicher Abstimmung abgelehnt.[4]"
4,5,Hans Klein,0.069,44,"1971 war er als NOK-Pressechef in Kurt Wilhelms Fernsehkomödie Olympia-Olympia neben Beppo Brem, Joachim Fuchsberger und Helga Anders zu sehen."


(4, 'Mittlere Reife und Schriftsetzerlehre', 5, 3688, 15, 2816)


In [14]:
dqr_predict, comment_predict, confidence_score, prompt_tokens, completion_tokens, cached_tokens = text_to_dqr("gpt-4.1", system_prompt, biography)